# Export participant data tabular formats

> Scope: Retrieve fields from phenotypic table and export them to CSV and XLSX files

- runtime: 10min 
- recommended instance: mem1_ssd1_v2_x8
- cost: <£0.10

This notebook depends on:
* **A Spark instance**

In this notebook, we will combine commands and routines from previous notebooks.
The goal is to find and retrieve selected fields from the phenotype database. 
Then we will export this data to CSV and Excel files - this is useful for further analyses and necessary for creating input files for tools like PLINK and regenie.

Further information can be found in Research Analysis Platform documentation: https://dnanexus.gitbook.io/uk-biobank-rap/working-on-the-research-analysis-platform/using-spark-to-analyze-tabular-data


## Import `dxdata` package and initialize Spark engine
### Docs at: https://github.com/dnanexus/OpenBio/blob/master/dxdata/getting_started_with_dxdata.ipynb

In [1]:
import dxdata
import pandas as pd
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

## Connect to the dataset

Next, we can set a `DATASET_ID` variable, which takes a value: `[projectID]:[dataset ID]`
We use it to define the `dataset` with `dxdata.load_dataset` function.

**projectID** and **dataset ID** values are unique to your project.
Notebook example **101** explains how to get them.

In [2]:
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)


## Retrieve data from the table

The following code selects the `participant` table.
Then we can define which field we are interested in using the `find_field` function.

There are three main ways to identify the field of interest:

- With `name` argument: here we give field ID. We can construct filed ID used by `dxdata` package from the field ID defined by UKB Showcase. The numeric showcase ID is translated to the Spark DB column name by adding the letter `p` at the beginning: e.g. *Standing height* showcase id is `50`, so Spark ID would be `p50`. Usually, fields have multiple instances. In such case, we add the `_i` suffix followed by instance number, e.g. *Standing height | Instance 0* will be `p50_i0`
- With `title` argument: here we define the filed by full title, followed by ` | Instance` suffix, e.g. `Age at recruitment` or `Standing height | Instance 0`
- With `title_regex` argument: here we define the filed by [regular expression](https://docs.python.org/3/howto/regex.html) matching the part of the title. We can use a keyword here, e.g. `.*height.*` will return all columns with the word *height* in the title.

In [3]:
pheno = dataset['participant']

# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

Next, we build a `field_list` array, comprising the fields we defined in our variables.

In [4]:
field_list = [field_eid, field_height, field_sex, field_age]
field_list

[<Field "eid">, <Field "p50_i0">, <Field "p31">, <Field "p21022">]

Function `retrieve_fields` build Spark DataFrame that reference data in the Spark database.
We pass three parameters: 
- `engine` - Spark engine defined before
- `fields` - here we provide our filed list
- `coding_values` - the `replace` parameter here means that a function will decode any codings and replace numeric codes with informative text labels

> Note: Decoding values might fail for complex, hierarchical coding, setting all values to `null`. In such a case set the parameter to `raw` to retrieve numeric codes, and then write an appropriate routine to convert these to text labels. See **Notebook 202** for an example.

In [6]:
pheno_data = pheno.retrieve_fields(engine=engine, fields=field_list, coding_values="replace")
pheno_data

DataFrame[eid: string, p50_i0: double, p31: string, p21022: bigint]

## Retrieve data to local memory

Calling function `toPandas()` on Spark DataFrame object will do the following;
- collect the values from the Spark cluster to local memory
- convert them to [Pandas](https://pandas.pydata.org/) data frame

In [7]:
data_tab = pheno_data.toPandas()

## Rename columns and preview the data

Column names in `data_tab` are set to field ID, e.g. `p50_i0` for height.
Before exporting the table we can rename the columns.

In [6]:
data_tab.columns = ["eid", "height", "sex", "age"]
data_tab

We can inspect the table. For example, function `head()` allows print out top 5 rows of data: `data_tab.head()`

## Export data to tabular text files 

Now, we can save the data for further analysis. `to_csv` saves data to comma-separated values file (NB `to_tdf` saves them to tab-delimited format, which is used by most external tools, for example, PLINK).

In [12]:
data_tab.to_csv('pheno_height_sex_age_500k.csv')

## Export data to Excel format 

Finally, we can export data to Excel XLSX format. First, we need to install `openpyxl`: a Python library that adds support for this format to Pandas. Next, we import it and run `to_excel`, which outputs data to XLSX format.

In [ ]:
%pip install openpyxl

In [11]:
import openpyxl

In [15]:
data_tab.to_excel('pheno_height_sex_age_500k.xlsx')

## Download the Excel and CSV files to your project

We will use this Excel for further analyses in **Notebook 203** 

In [ ]:
%%bash
dx mkdir pheno
dx upload pheno_height_sex_age_500k.* --path pheno/